In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import math
import joblib

# 加载模型
model = joblib.load('rf_model.joblib')
feature_cols = joblib.load('rf_features.joblib')

def predict_energy(total_floor_area, estimated_floor_count, epc_score,
                   wall_insulation, roof_type, roof_insulation, glazing_type,
                   built_form, main_heat_type, main_fuel_type, lookup_age_band, wall_type):
    U_wall = 0.37 if wall_insulation == 'insulated' else 1.7
    U_roof = 0.25 if roof_insulation == 'insulated' else (0.28 if roof_type == 'flat' else 2.3)
    U_floor = 0.25
    U_glazing = 2.4 if glazing_type == 'double/triple' else (2.82 if glazing_type == 'secondary' else 5.75)
    if roof_type == 'pitched':
        roof_area = total_floor_area / math.cos(math.radians(30))
    elif roof_type == 'flat':
        roof_area = total_floor_area
    else:
        roof_area = 0
    wall_area = total_floor_area * 2.1
    floor_area = total_floor_area
    glazing_area = total_floor_area * 0.18
    delta_T = 18

    Q_total = (
        U_wall * wall_area +
        U_roof * roof_area +
        U_floor * floor_area +
        U_glazing * glazing_area
    ) * delta_T

    rowdict = {
        'epc_score': epc_score,
        'estimated_floor_count': estimated_floor_count,
        'wall_area': wall_area, 'roof_area': roof_area, 'floor_area': floor_area, 'glazing_area': glazing_area,
        'u_value_wall': U_wall, 'u_value_roof': U_roof, 'u_value_floor': U_floor, 'u_value_glazing': U_glazing,
        'Q_total': Q_total,
        'wall_type': wall_type, 'wall_insulation': wall_insulation,
        'roof_type': roof_type, 'roof_insulation': roof_insulation, 'glazing_type': glazing_type,
        'built_form': built_form, 'main_heat_type': main_heat_type,
        'main_fuel_type': main_fuel_type, 'lookup_age_band': lookup_age_band
    }
    df_input = pd.DataFrame([rowdict])
    df_input = pd.get_dummies(df_input)
    for c in feature_cols:
        if c not in df_input.columns:
            df_input[c] = 0
    df_input = df_input[feature_cols]
    pred = model.predict(df_input)[0]
    return f"预测年能耗约为: {pred:,.0f} kWh\n主物理热损失Q_total为: {Q_total:,.1f}"

# Gradio界面
iface = gr.Interface(
    fn=predict_energy,
    inputs=[
        gr.Number(label="总建筑面积 (m²)", value=60),
        gr.Number(label="地上楼层数", value=4),
        gr.Number(label="EPC得分", value=70),
        gr.Radio(['insulated', 'uninsulated'], label="墙体是否保温"),
        gr.Radio(['pitched', 'flat', 'room in roof', 'another dwelling above'], label="屋顶类型"),
        gr.Radio(['insulated', 'uninsulated', 'room in roof', 'another dwelling above'], label="屋顶是否保温"),
        gr.Radio(['single/partial', 'double/triple', 'secondary'], label="窗户类型"),
        gr.Radio(['end-terrace','mid-terrace'], label="房屋结构"),
        gr.Radio(['boiler', 'communal', 'room/storage heaters', 'heat pump', 'other', 'no heating system'], label="供热类型"),
        gr.Radio(['mains gas', 'electricity', 'other', 'no heating system'], label="主要燃料"),
        gr.Radio(['pre-1920', '1930-1949', '1950-1966', '1967-1982', '1983-1995', '1996-2011', '2012-onwards'], label="年代"),
        gr.Radio(['solid','cavity'], label="墙体类型")
    ],
    outputs=gr.Textbox(label="预测结果"),
    title="能耗灰盒预测工具"
)

iface.launch(share=True)  # 这样结束后会自动给你一个可点击的公网链接

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://4e24912b8c989601a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
